# import lib and test GPU of tensorflow


In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
# import matplotlib.image as mping
import matplotlib.animation as animation
import time, datetime
import sys
import gc
import os
import tensorflow as tf
import h5py
from random import randint
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import manifold, datasets
from IPython.display import clear_output
from tsnecuda import TSNE
#import tsnecuda
#tsnecuda.test()


In [ ]:
os.environ["QT_API"] = "pyqt5"
print(os.environ.get('QT_API'))


In [ ]:
%matplotlib qt


In [ ]:
#test tesnsorflow
print("tensorflow version: ",tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))
# Returns whether TensorFlow was built with CUDA (GPU) support.
print("tf.test.is_Built_With_cuda = ",tf.test.is_built_with_cuda())
# RetUrns whether TensorFlow can access a GPU. (deprecated)
print("tf.test.is_gpu_available = ",tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))


# def function


In [ ]:
def show_picture(img, figsizeX=6, figsizeY=4):
    # plt.figure(figsize=(figsizeY, figsizeX))
    plt.axis("off")
    # plt.imshow(img, plt.cm.gray, vmax=1.0)
    plt.imshow(img, plt.cm.gray)


# Main program -------------->
# build data path index


In [ ]:
def build_data_path_index():
    print("build data path index.....")
    ## read file path
    # folder_path = 'H:/mushroom/VL/Mat/*'                                             # Tommy windows
    # folder_path = '/media/tommy/本機磁碟1/mushroom/VL/Mat/*'                          # Tommy Linux
    folder_path = '/home/shawn/DATA/Special_topic/Pleurotus_eryngii_complete/VL/Mat/*' # Shawn Linux
    folder_path_list = sorted(glob.glob(folder_path))[1:] #excpet 20200507A
    # print('\n'.join(folder_path_list))
    file_name_p = []
    for path in folder_path_list: #folder_name_i = folder_name_index
        file_name_p += sorted(glob.glob(path+'/*'))
        # break

    file_name = [[],[]]
    # category string
    # CS=37 #Tommy Windows
    # CS=53 #Tommy Linux
    CS=83 #Shawn Linux
    for name in file_name_p:
        if name[CS] == 'A': #change to your index
            file_name[0].append(name)
        elif name[CS] == 'B':
            file_name[1].append(name)

    print("build successful")
    return file_name, CS


In [ ]:
# file_name : file_name[0] -----> rank of A
#             file_name[1] -----> rank of B
file_name, CS = build_data_path_index()

print()
print("Rank A of number is ", len(file_name[0]))
print("Rank B of number is ", len(file_name[1]))
TFnum = len(file_name[0]) +len(file_name[1]) # Total file number
print("total data number is ", TFnum)


# del data to picture to observe Placement method


# define function of Process data


In [ ]:
def select_U_value():
    # select undesired or interfering signatures
    U =               cube_arr[:,305,284]
    U = np.vstack((U, cube_arr[:,680,440]))

    U = np.transpose(U)
    np.save('/home/shawn/DATA/Special_topic/program/Pleurotus_eryngii_complete/U_value.npy',U)
    print("U.shpae = ", U.shape)

def OSP_preprocess(data, select=70):
    ## change data(band, y, x)to
    ##        cube_arr([band][y*x])

    ## $select how band will input to OSP

    cube_arr = data[0]
    cube_arr = cube_arr[np.newaxis,:]
    (band, x, y) = data.shape
    step = band/select
    index = -1
    for i in range(select):
        index = index+step
        # print(index)
        cube_arr = np.concatenate((cube_arr, [data[round(index)]]), axis=0)

    ## if youe need to reselect U_value uncomment it
    # select_U_value(cube_arr)

    (band, x, y) = cube_arr.shape
    cube_arr = cube_arr.reshape(band, x*y)
    return cube_arr


def rho(arr):
    sum=0
    for i in arr:
        sum += i**2
    r = sum**0.5
    return r
def OSPfn(M, U):
    (l, p) = M.shape
    P_U = np.identity(l) - np.dot(U,np.linalg.pinv(U));
    nu = np.zeros((p,1))
    for i in range(p):
        nu[i] = rho(P_U.dot(M[:,i]))
    return nu

def read_data(file_name, crop_x, crop_y, i, j):
    temp = h5py.File(file_name[i][j], 'r').get('cube')
    data = np.array(temp)
    data = data[:, :crop_y, :crop_x]
    return data

def get_contours(file_name, osp_out, data, crop_x, crop_y, CS, i, j):

    NumOPE = int(file_name[i][j][-6:-4]) - int(file_name[i][j][-9:-7]) +1 # number of Pleurotus eryngii in this cube
    mask=(osp_out/np.max(osp_out))>0.4

    KV = 9 # kernel Value
    while(True): # prevent KV too low to let mask quantity higher or less then PE
        # print("KV = ", KV)
        blurred =cv2.medianBlur(np.float32(mask), 5)
        kernel = np.ones((KV,KV),np.uint8)
        blurred=cv2.morphologyEx(blurred, cv2.MORPH_OPEN, kernel)

        blurred = np.array(blurred,np.uint8)
        blurred=np.clip(blurred, 0, 255)
        contours, _ = cv2.findContours(blurred, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        if(KV >= 35):
            build_mask_and_save(data, contours, crop_x, crop_y, CS, i, j)
            print('\n\t\t\t\ti={},j={},KV={}, len(contours)={}, NumOPE={}'.format(i,j,KV, len(contours), NumOPE), end="\t\t\t\t")
            return 0
            # raise UserWarning('i={},j={},KV={}, len(contours)={}, NumOPE={}'.format(i,j,KV, len(contours), NumOPE))
        if(len(contours) != NumOPE): # prevent KV too low to let mask quantity higher or less then PE
            KV += 2
        else:
            break
    return contours

def build_mask_and_save(data, contours, crop_x, crop_y, CS, i, j):

    plt.clf()
    plt.subplot(2,6,1)
    plt.title(file_name[i][j][CS-4:])
    show_picture(data[150])

    temp = np.copy(data[150])
    temp[temp<0] = 0
    temp *= (255/np.max(temp))
    # cv2.imwrite('/home/shawn/DATA/Special_topic/Pleurotus_eryngii_complete/mask/{}.png'.format(file_name[i][j][CS-4:-4]),temp)

    for contours_i in range(len(contours)):
        clone=np.zeros([crop_y,crop_x],dtype=np.int16)
        clone=cv2.drawContours(clone, [contours[contours_i]], contourIdx=-1, color=(255,255,255), thickness=-1)
        clone//=255

        plt.subplot(2,6,contours_i+2)
        plt.title(file_name[i][j][CS-4:-4]+"_{:d}".format(contours_i))
        show_picture(clone)

        (band, x, y) = data.shape

        ## calculate mean of band for all PE and save it
        spectral_vaule_arr = np.zeros(300)
        for band_i in range(band):
            spectral_vaule_arr[band_i] = np.sum(data[band_i]*clone)/np.count_nonzero(clone)
        path = "/home/shawn/DATA/Special_topic/Pleurotus_eryngii_complete/spectral_curve/"+file_name[i][j][CS-4:-4]+"_{:d}.npy".format(contours_i)
        np.save(path, spectral_vaule_arr)

        plt.subplot(2,6,contours_i+8)
        plt.title(file_name[i][j][CS-4:-4]+"_{:d}".format(contours_i))
        plt.plot(np.arange(0,300,dtype=np.int16), spectral_vaule_arr)

        plt.savefig('/home/shawn/DATA/Special_topic/Pleurotus_eryngii_complete/mask/{}.png'.format(file_name[i][j][CS-4:-4]))



In [ ]:
def process_data(TFnum, file_name, CS, U, crop_x, crop_y, i, j, fault_count):
    # U = np.load('/home/shawn/DATA/Special_topic/program/Pleurotus_eryngii_complete/U_value.npy')
    # crop_x = 850
    # crop_y = 820
    # ST = datetime.datetime.now()
    # print("processing.... Start time =",ST.strftime("%H:%M:%S"))

    print("i={}/2 j={:2d}/{}".format(i+1,j+1,len(file_name[i])),end="\t") # display progress rate
    ST = datetime.datetime.now()

    ### ========== read data ==========
    print("read_{}".format(file_name[i][j][CS-4:]),end="\t")
    data = read_data(file_name, crop_x, crop_y, i, j)

    ### ========== OSP data ==========
    ## if you change value of select which in OSP_preprocess(), you must to reset U_value.npy manually
    print("OSP", end="\t")
    cube_arr = OSP_preprocess(data)
    osp_out = OSPfn(cube_arr, U).reshape((crop_y,crop_x))

    ### ========== get contours ==========
    print("get_contours", end="\t")
    contours = get_contours(file_name, osp_out, data, crop_x, crop_y, CS, i, j)

    if contours==0:
        fault_count += 1
    else:
        ### ========== build mask_and save ==========
        print("build_mask_and_save", end="\t")
        build_mask_and_save(data, contours, crop_x, crop_y, CS, i, j)

    ET = datetime.datetime.now()
    print("ET:",ET.strftime("%H:%M:%S"),end='\t')
    print("Exec.:",ET-ST, end='\t')
    print("ETC:",(ST+(ET-ST)*(TFnum)).strftime("%H:%M:%S"))
    TFnum -=1
    return fault_count, TFnum




# Process Data


In [ ]:
# fullscreen fig manually
plt.subplot(1,1,1)
plt.show()


In [ ]:
if __name__ == '__main__':
    i_start = 0
    j_start = 0

    U = np.load('/home/shawn/DATA/Special_topic/program/Pleurotus_eryngii_complete/U_value.npy')
    crop_x = 850
    crop_y = 820
    ST = datetime.datetime.now()
    print("processing.... Start time =",ST.strftime("%H:%M:%S"))

    fault_count = 0
    for i in range(i_start, 2):
        for j in range(j_start, len(file_name[i])):
            fault_count, TFnum = process_data(TFnum, file_name, CS, U, crop_x, crop_y, i, j, fault_count)
            plt.show()

    print("\nprocessing successful")
    print('fault_count = ', fault_count)


# test


In [ ]:
## test npy
arr = np.load(path)
plt.plot(np.arange(0,300,dtype=np.int16), arr)
